In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import seaborn

In [7]:
artists = pd.read_csv("artists.csv")
artworks = pd.read_csv("artworks.csv")

In [8]:
artworks = artworks.set_index(artworks['Artwork ID'])
artists = artists.set_index(artists['Artist ID'])

In [11]:
def find_year(datestr):
    if datestr is np.nan:
        return None
    match = re.search('[1-2]\d\d\d', datestr)
    if match is None:
        return -1
    else:
        return int(match.group())

In [12]:
artworks['Year'] = artworks['Date'].apply(find_year)

In [17]:
workcount_arr = np.zeros(artists['Artist ID'].max()+1, dtype=int)
for i, s in artworks['Artist ID'].iteritems():
    if s is np.nan:
        continue
    for j in [(x.strip()) for x in s.split(',')]:
        if j == '': continue
        try:
            workcount_arr[int(j)] += 1
        except ValueError:
            print 'Invalid id %s' % s
workcount = pd.Series(workcount_arr, name='Work Count')
artists = artists.join(workcount, on='Artist ID')

In [18]:
collaboration = []
for i, s in artworks['Artist ID'].iteritems():
    if s is np.nan or s == '':
        continue
    if len(s.split(',')) > 1:
        collaboration.append([int(x.strip()) for x in s.split(',')])

In [19]:
len(collaboration), len(artists)

(6429, 15091)

In [45]:
collab = []
collab_id = []
weight = []
for ind, r in artworks.iterrows():
    s = r['Artist ID']
    if s is np.nan or len(s.split(',')) <= 1:
        continue
    collaborators = np.array([int(x.strip()) for x in s.split(',')])
    collaborators = np.unique(collaborators)
    # use work count as proxy for importance
    w = artists['Work Count'][collaborators].sum()
    for i in range(len(collaborators)):
        for j in range(i+1, len(collaborators)):
            collab.append(set((collaborators[i],collaborators[j])))
            collab_id.append(r['Artwork ID'])
            weight.append(w)

In [21]:
len(collab)

45397

In [52]:
artist1 = np.array(map(lambda x: list(x)[0], collab))
artist2 = np.array(map(lambda x: list(x)[1], collab))
edges = pd.DataFrame(np.hstack([artist1[:,np.newaxis], artist2[:,np.newaxis], 
                     np.array(collab_id)[:,np.newaxis],
                     np.array(weight)[:,np.newaxis]]),
                     columns = ['Source', 'Target', 'Artwork ID', 'Weight'])

In [53]:
edges = edges.join(artworks['Title'], on=['Artwork ID'])

In [60]:
edges[['Source', 'Target', 'Weight', 'Title']].to_csv('collab_edges.csv', encoding='utf-8')

In [61]:
artists[['Name', 'Birth Year', 'Work Count']].to_csv('collab_nodes.csv', encoding='utf-8')